In [41]:
import matplotlib.pyplot as plt
import snapatac2 as snap
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import silhouette_score
import glob
import warnings
import os
warnings.filterwarnings("ignore")


In [42]:
files_l3s = glob.glob("/data2st1/junyi/output/atac0416/iterative/l3/*.csv")

In [43]:
df_l3= pd.DataFrame()
for file in files_l3s:
    if file.find("summary.csv")>0:
        continue

    # Read the CSV file into a DataFrame
    df = pd.read_csv(file, index_col=0)
    if len(df_l3)==0:
        df_l3 = df
    else:
        df_l3 = pd.concat([df_l3, df], axis=0)

In [44]:
df_l3['celltype.L2']= df_l3['celltype.L3'].str[:-2]

In [45]:
df_l3.groupby(['celltype.L2']).nunique().to_csv(".csv")

In [ ]:
# df_meta_all = pd.read_csv('/data2st1/junyi/output/atac0416/ATACSC_3REGION_ALL_L2annoated.csv',index_col=0)

# # %%
# black_list = ['Immune','OPC-Oligo','Doublet','PFC Doublet','PFC Not sure','Not sure','Astro-Epen','Neuron']

# # %%
# # df_meta_all["celltype.L2"].unique()
# # l2type = df_meta_all["celltype.L2"].unique()[0]

# # %%
# best_ress = []
# n_clusters = []
# n_cells = []
# ctypes = []

# for l2type in df_meta_all["celltype.L2"].unique():
#     if l2type in black_list:
#         continue
#     print(l2type)

AMY Meis1 Glut
AMY Egfr GABA
AMY Gad2 GABA
AMY Slc17a6 Glut
AMY Tafa1 Glut
AMY Adamtsl1 Glut
AMY Dach1 Glut
AMY Sox6 GABA
AMY Sim1 Glut
AMY Foxp2 GABA
AMY Zfp536 GABA
AMY Tshz3 Glut
AMY Pde3a GABA
Hippo DG GC
Hippo Mossy cell
Hippo CA3-d pyramidal
Hippo Lamp5 GABA
Cell segment
Hippo CA3-v pyramidal
Hippo CA1-d pyramidal
Hippo CA2 pyramidal
Hippo Pvalb GABA
Hippo Htr3a GABA
Hippo Sst GABA
Hippo Vip GABA
Hippo CA1-v pyramidal
Hippo Subiculum
PFC L6 CT
PFC L5 ET
PFC L4/5 IT
PFC L2/3 IT
PFC L6 IT
PFC NP
PFC Pvalb GABA
PFC Sst GABA
PFC Vip GABA
PFC L6b
PFC Lamp5 GABA
PFC Car3
PFC Pvalb Vipr2 GABA
PFC Sncg GABA
PFC Sst Chodl GABA


In [ ]:
best_ress = []
n_clusters = []
n_cells = []
ctypes = []
for file_l3 in files_l3s:
    max_silhoutte = 0
    best_res = 0
    if file_l3.find("summary.csv")>0:
        continue
    df_l3 = pd.read_csv(file_l3,index_col=0)
    df_l3_count = df_l3.value_counts().reset_index()
    ctname = os.path.basename(file_l3).split(".")[0].replace("_l3","")
    if ctname.find("PFC")>0:
        continue
    datasubset = snap.read_dataset(f'/data2st1/junyi/output/atac0416/iterative/l3/{ctname}.h5ads/_dataset.h5ads')
                # Iterate over different resolution values
    X=datasubset.obsm['X_spectral']
    for res in np.arange(0.1, 2.0, 0.1):
        snap.tl.leiden(datasubset, resolution=res)
        Y=datasubset.obs['leiden']

        # Calculate silhouette score
        try:
            score = silhouette_score(X,Y)
        except ValueError as e:
            score = 0
        if score > max_silhoutte:
            max_silhoutte = score
            best_res = res
            print(f"Best silhouette score: {max_silhoutte} at resolution {best_res}")

        # if len(df_plot) == 0:
        #     df_plot = pd.DataFrame({"UMAP1":datasubset.obsm['X_umap'][:,0],"UMAP2":datasubset.obsm['X_umap'][:,1],
        #                             "resolution":np.round(res,1),"silhouette_score":score,
        #                             "leiden":Y})

        # else:
        #     df_tmp = pd.DataFrame({"UMAP1":datasubset.obsm['X_umap'][:,0],"UMAP2":datasubset.obsm['X_umap'][:,1],
        #                             "resolution":np.round(res,1),"silhouette_score":score,
        #                             "leiden":Y})
        #     df_plot = pd.concat([df_plot,df_tmp],ignore_index=True)
        
        # Stop if the median cluster size is less than 400
        if Y.to_pandas().value_counts().median()<400:
            print(f"Resolution {res} has a max cluster with less than 400 cells")
            break

        # Plot the results
        snap.tl.leiden(datasubset, resolution=best_res)
        # sns.relplot(
        #     data=df_plot, x="UMAP1", y="UMAP2",
        #     col="resolution", hue="leiden",
        #     col_wrap=5,linewidth=0,
        #     height=3, aspect=1.3
        # )
    celltype = ctname
    celltype_save = celltype.replace(" ","_")
    celltype_save = celltype_save.replace("/","-")
    # plt.savefig(f'/data2st1/junyi/output/atac0416/iterative/l4/{celltype_save}_umap.png', dpi=300, bbox_inches='tight')
    df_meta_subtypes = pd.DataFrame({"celltype.L3":datasubset.obs['leiden'].to_numpy()},index=datasubset.obs_names)
    df_meta_subtypes["celltype.L3"] = celltype+"-"+df_meta_subtypes["celltype.L3"].astype(str)
    # df_meta_subtypes.to_csv(f'/data2st1/junyi/output/atac0416/iterative/l4/{celltype_save}_l4.csv')
    best_ress.append(best_res)
    n_clusters.append(len(datasubset.obs['leiden'].unique()))
    ctypes.append(celltype)
    n_cells.append(len(datasubset.obs_names))

Best silhouette score: 0 at resolution 0.1
Best silhouette score: 0.1198569956240497 at resolution 0.4
Resolution 0.4 has a max cluster with less than 400 cells
Best silhouette score: 0 at resolution 0.1
Best silhouette score: 0.11097915371754728 at resolution 0.4
Resolution 0.5 has a max cluster with less than 400 cells
Best silhouette score: 0 at resolution 0.1
Best silhouette score: 0.09227241085972886 at resolution 0.4
Resolution 0.5 has a max cluster with less than 400 cells
Best silhouette score: 0 at resolution 0.1
Best silhouette score: 0.16934880987220818 at resolution 0.2
Best silhouette score: 0.17083706929761508 at resolution 0.30000000000000004
Resolution 1.9000000000000001 has a max cluster with less than 400 cells
Best silhouette score: 0 at resolution 0.1
Best silhouette score: 0.09695813489748309 at resolution 0.30000000000000004
Resolution 0.5 has a max cluster with less than 400 cells
Best silhouette score: 0 at resolution 0.1
Resolution 0.1 has a max cluster with le

In [39]:
df_summary = pd.DataFrame({"celltype.L3":ctypes,"best_resolution":best_ress,"n_clusters":n_clusters,"n_cells":n_cells})  

In [40]:
df_summary

,celltype.L3,best_resolution,n_clusters,n_cells
0,Hippo_Pvalb_GABA,0.4,2,778
1,PFC_Lamp5_GABA,0.4,3,986
2,AMY_Zfp536_GABA,0.4,4,1193
3,AMY_Slc17a6_Glut,0.3,17,6566
4,PFC_Vip_GABA,0.3,3,1151
5,PFC_L6b,0.1,1,346
6,PFC_L2-3_IT,0.2,3,14389
7,Cell_segment,0.4,3,966
8,AMY_Foxp2_GABA,0.4,8,2111
9,Hippo_Mossy_cell,0.2,10,4225


In [8]:
df_summary.to_csv('/data2st1/junyi/output/atac0416/iterative/l4/l4_summary.csv')